Formulation of problem given in exercis 3 as LPP

To formulate and solve this problem let us divide quantity for each of the the 10 available alloys in two parts 
1. quantity to be used form inhouse stock, xi 
2. quantity to be purchased,yi

Given
1.total product qty = 250 tons  
Composition  
45% Chromium  
35% zinc  
20% silver  

% compostion of all the 10 alloys and their inhouse stock (tons) is given.
Also cost for using inhouse stock and cost of procuring for each alloys is given.

Using these data our formulation is

Objective(minimise)
x1 * 35+y1 * 72 + x2 * 50+y * 95 + x3 * 58+y3 * 110 + x4 * 60+y4* 125 + x5* 44+y5* 88 + x6* 39+y6* 74 + x7* 45+y7* 95 + x8* 55+y8* 115 + x9* 35+y9* 60 + x10* 40+y10* 84

Such that  
x1<=10  
x2<=7  
x3<=8  
x4<=15  
x5<=8  
x6<=15  
x7<=12  
x8<=10  
x9<=0  
x10<=9

x1+x2+x3+x4+x5+x6+x7+x8+x9+x10+y1+y2+y3+y4+y5+y6+y7+y8+y9+y10=250

(x1*0.8+x2*0.75+x3*0.75+x4*06+x5*0.55+x6*0.55+x7*0.4+x8*03.5+x9*0.3+x10*0.3+y1*0.8+y2*0.75+y3*0.75+y4*0.6+y5*0.55+y6*0.55+y7*0.4+y8*0.35+y9*0.3+y10*03)=0.45*250

(x1*0.15+x2*0.15+x3*0.10+x4*20+x5*0.25+x6*0.10+x7*0.5+x8*0.15+x9*0.3+x10*0.55+y1*0.15+y2*0.15+y3*0.10+y4*0.20+y5*0.25+y6*0.10+y7*0.50+y8*0.15+y9*0.3+y10*0.55)=0.35*250

(x1*0.05+x2*0.10+x3*0.15+x4*0.20+x5*0.20+x6*0.35+x7*0.10+x8*0.50+x9*0.40+x10*0.15+y1*0.05+y2*0.10+y3*0.15+y4*0.20+y5*0.20+y6*0.35+y7*0.10+y8*0.50+y9*0.40+y10*0.15)=0.20*250


In [ ]:
!pip install pyomo


In [ ]:
!apt-get install -y -qq glpk-utils

In [3]:
from pyomo.environ import*
import numpy as np

#model creation
model=ConcreteModel()

# M is value of available alloys and N is value of constraints
N=10
M=4

#constraint coefficients 
constraint_coef_A=np.array([[1,1,1,1,1,1,1,1,1,1],
                            [0.80,0.75,0.75,0.60,0.55,0.55,0.40,0.35,0.30,0.30],
                            [0.15,0.15,0.10,0.20,0.25,0.10,0.50,0.15,0.30,0.55],
                            [0.05,0.10,0.15,0.20,0.20,0.35,0.10,0.50,0.40,0.15]])

#constraint rhs values
constraint_rhs_b=np.array([250,0.45*250,0.35*250,0.20*250])


#objective function array is 10x2 array for storing in house price and pruocuring cost
obj_coef_c=np.array([[35,50,58,60,44,39,45,55,35,40],
                     [72,95,110,125,88,74,95,115,60,84]])

#bounds for decision variables
inhouse_lower_bound=np.array([0,0,0,0,0,0,0,0,0,0])
inhouse_upper_bound=np.array([10,7,8,15,8,15,12,10,0,9])
procured_lower_bound=np.array([0,0,0,0,0,0,0,0,0,0,])
procured_upper_bound=np.array([np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf,np.inf])

#list of index for traverisng arrays
col_indices=np.arange(N)
row_indices=np.arange(M)

#variable declaration
model.inhouse_qty=Var(col_indices)
model.procured_qty=Var(col_indices)

#empty list creation for containing constraints
model.constraints=ConstraintList()

#constraints addition to model
for i in row_indices:
    model.constraints.add(sum(model.inhouse_qty[j]*constraint_coef_A[i,j]+model.procured_qty[j]*constraint_coef_A[i,j] for j in col_indices)==constraint_rhs_b[i])

#setting variables bounds
for j in col_indices:
    model.inhouse_qty[j].setlb(inhouse_lower_bound[j])
    model.inhouse_qty[j].setub(inhouse_upper_bound[j])
    model.procured_qty[j].setlb(procured_lower_bound[j])
    model.procured_qty[j].setub(procured_upper_bound[j])

#objective function 
model.objective=Objective(expr=sum(model.inhouse_qty[j]*obj_coef_c[0,j]+model.procured_qty[j]*obj_coef_c[1,j] for j in col_indices),sense=minimize)     

#printing model to check it is correctly created
model.pprint()


3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    inhouse_qty_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    procured_qty_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

2 Var Declarations
    inhouse_qty : Size=10, Index=inhouse_qty_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :    10 : False :  True :  Reals
          1 :     0 :  None :     7 : False :  True :  Reals
          2 :     0 :  None :     8 : False :  True :  Reals
          3 :     0 :  None :    15 : False :  True :  Reals
          4 :     0 :  None :     8 : False :  True :  Reals
          5 :     0 :  None :   

**Model has been created as required**

In [4]:
#optimising model and writing solver status
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16238.44
  Upper bound: 16238.44
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 21
  Number of nonzeros: 81
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.012109756469726562
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

In [11]:
print('\noptimal cost = ', model.objective())
print('\nquantity of alloy at optimum\n')
for i in col_indices:
   print('procured qty of alloy',i+1,'=',model.procured_qty[i].value,'kgs')
   print('inhouse qty of alloy',i+1,'=',model.inhouse_qty[i].value,'kgs')
   print("\n")




optimal cost =  16238.44

quantity of alloy at optimum

procured qty of alloy 1 = 28.6 kgs
inhouse qty of alloy 1 = 10.0 kgs


procured qty of alloy 2 = 0.0 kgs
inhouse qty of alloy 2 = 7.0 kgs


procured qty of alloy 3 = 0.0 kgs
inhouse qty of alloy 3 = 8.0 kgs


procured qty of alloy 4 = 0.0 kgs
inhouse qty of alloy 4 = 15.0 kgs


procured qty of alloy 5 = 0.0 kgs
inhouse qty of alloy 5 = 8.0 kgs


procured qty of alloy 6 = 0.0 kgs
inhouse qty of alloy 6 = 15.0 kgs


procured qty of alloy 7 = 0.0 kgs
inhouse qty of alloy 7 = 12.0 kgs


procured qty of alloy 8 = 0.0 kgs
inhouse qty of alloy 8 = 0.0 kgs


procured qty of alloy 9 = 52.64 kgs
inhouse qty of alloy 9 = 0.0 kgs


procured qty of alloy 10 = 84.76 kgs
inhouse qty of alloy 10 = 9.0 kgs




Ans.4 
There are only three alloys which are needed to be bought and quantites are 
alloy 10 = 84.76 kgs  
alloy 9 = 52.64 kgs   
alloy 1 = 28.6 kgs

Since there are only three alloys to be bought so they can be ranked in both ways, either high to low or low to high in terms of purchase quantity.

Ans.5
Except alloy8 and alloy9 stock of all other alloys are completely used, this is so because their required quantity at optimum is more than that available in the stock.

Ans.6
Except alloy 1,9 and 10, no other alloys is needed to be acquired from market because their requird quantity at optimum is either less or same as the stocked quantity.



